# Programming and Database Fundamentals for Data Scientists - EAS503

## A practical introduction to IPython Notebooks
## Some notes about installation:

### Don't do this:

```
sudo apt-get install ipython-notebook
```

### Instead, do this:

```
pip install ipython tornado pyzmq
```


Otherwise, follow the instructions at: [https://docs.continuum.io/anaconda/install/](https://docs.continuum.io/anaconda/install/)

You can start IPython notebook by running

```
ipython notebook --pylab inline
```


### Demonstrating a simple data science pipeline.
Data available from [Chicago Crime Data](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2)

In [1]:
import MySQLdb

In [5]:
# Start the connection to database
# make sure to close it after your are done
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="root",       # your password
                     db="eas503db")    # name of the data base

In [11]:
querystr = 'SELECT count(*) from chicago_crime_data where '
cur = db.cursor()
cur.execute(querystr)

InterfaceError: (0, '')

In [9]:
for row in cur.fetchall():
    print(row)

(6414902,)


In [12]:
db.close()

ProgrammingError: closing a closed connection